<a href="https://colab.research.google.com/github/sauravaa/Bootstrap/blob/main/hill_climb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Submitted By:
# Saurav Anand(2402res17)
#Course-E.Mtech(CSE)
#Sem- 1st
#ASSIGNMENT-3: Hill Climb

In [8]:
import numpy as np
import random

In [9]:

def p_print(x):                              # fun. to print list as 3*3 puzzle format
  print(f" {x[0]} | {x[1]} | {x[2]}" )
  print("-----------")
  print(f" {x[3]} | {x[4]} | {x[5]}" )
  print("-----------")
  print(f" {x[6]} | {x[7]} | {x[8]}", '\n')

def b_to_zero(x):                            #fun. to convert 'B' in to zero bcz it will create broblem while calculating hash value.
  for i in range(len(x)):
    if x[i]=='B':
       x[i]=0
  return x

def is_solvable(matrix):
  ''' Count the numbers of inversion.An inversion occurs when a higher-numbered tile appears before a lower-numbered tile in the puzzle.
  If the total number of inversions is even, the puzzle may be solvable.Let's take an example,

5|2|8                     5 precedes 1,2,3,4 - 4 inversions
-----
4|1|7                     2 precedes 1 - 1 inversion
-----                     8 precedes 1,3,4,6,7 - 5 inversions
 |3|6                     4 precedes 1,3 - 2 inversions
                          1 precedes none - 0 inversions
                          7 precedes 3,6 - 2 inversions
                          3 precedes none - 0 inversions
                          6 precedes none - 0 inversions
total inversions 4+1+5+2+0+2+0+0 = 14 (Even Number) So this puzzle is solvable.'''

  inv=0
  for i in range(len(matrix)):
    for j in range(i+1,len(matrix)):
      if matrix[j]!=0 and matrix[i]>matrix[j]:
        inv +=1
  if inv %2 ==0:
    return "Yes"
  else:
    return "No"

In [10]:
def find_position(matrix,val):
  '''return i and j of the val.'''
  return np.where(matrix==val)

def MOVE(current_state,move,i,j):
  temp_matrix=current_state.copy()
  temp_val=temp_matrix[i,j]
  if move=="RIGHT":
    temp_matrix[i,j]=temp_matrix[i,j+1]
    temp_matrix[i,j+1]=temp_val
  elif move=="LEFT":
    temp_matrix[i,j]=temp_matrix[i,j-1]
    temp_matrix[i,j-1]=temp_val
  elif move=="UP":
    temp_matrix[i,j]=temp_matrix[i-1,j]
    temp_matrix[i-1,j]=temp_val
  elif move=="DOWN":
    temp_matrix[i,j]=temp_matrix[i+1,j]
    temp_matrix[i+1,j]=temp_val
  new_state=temp_matrix
  return new_state

def neighbor_state(current_state):
  ''' This fun get the neighbor state of the current state.The neighbor will come after movement of blank tile
         in left,right,up or down movement.'''
  move_list=[("LEFT",0,-1),("RIGHT",0,1),("UP",-1,0),("DOWN",1,0)]
  row,col=find_position(current_state,0)
  max_row,max_column=current_state.shape
  neighbors=[]

  for move,i_offset,j_offset in move_list:
    new_i=row + i_offset
    new_j=col + j_offset
    if(0<=new_i<max_row)and (0<=new_j<max_column):
      new_state=MOVE(current_state,move,row,col)
      neighbors.append(new_state)
  return neighbors



In [11]:
def displaced_tile(current_state,goal_state):   # h(n) = finds the number of tiles displaced in current state from its position in goal state
  h=0
  max_row,max_column=current_state.shape
  for i in range(max_row):
    for j in range(max_column):
      if (current_state[i,j]!=0):
        if current_state[i,j] != goal_state[i,j]:
          h+=1
  return h

def manhattan_distance(current_state,goal_state):    # h(n) = finds sum of the Manhattan distance of each tile from the goal position.
  h=0
  max_row,max_col=current_state.shape
  for k in range(1,9):
    current_row,current_col=find_position(current_state,k)
    goal_row,goal_col=find_position(goal_state,k)
    h=h+abs((current_row-goal_row)+(current_col-goal_col))
  return h
#print(manhattan_distance(start_state,goal_state))

def selected_heuristic_distance(current_state,goal_state,Heuristic):
  if Heuristic=='h1':
    Distance=displaced_tile(current_state,goal_state)
  elif Heuristic=='h2':
    Distance=manhattan_distance(current_state,goal_state)
  return Distance


In [12]:
'''A local search algorithm tries to find the optimal solution by exploring the states inthe local region.
   Hill climbing is a local search technique that constantly looks for a better solution in its neighborhood. '''

def hill_climbing(start_state,goal_state,Heuristic):
  current_state=start_state
  current_distance=selected_heuristic_distance(current_state,goal_state,Heuristic)
  distances=[]
  state_explored=[]
  flag=0
  while flag==0:
    state_explored.append(current_state)
    if np.all(current_state==goal_state):
      '''print("SUCCESS")
      print('Total no. of State Explored : ',len(state_explored))
      print('Final Path is:')
      for k in state_explored:
        print(k,'\n') '''
      flag=1                                        # If reach the goal state set the flag=1
      break

    neighbors=neighbor_state(current_state)
    for i in neighbors:
      distance=selected_heuristic_distance(i,goal_state,Heuristic)
      distances.append(distance)

    min_distance=min(distances)
    ''' If the cost of minimum_cost neighbor is less then the cost of current state then replace the current state with that neighbor.
       (means change the current state with better neighbor if we have any else terminate)'''
    if min_distance < current_distance:
      current_distance=min_distance
      k=np.argmin(distances)                       #find the index where list distances have min. value
      current_state=neighbors[k]                   #neighbors and distances list are of same size so the index are also same
      distances.clear()

    else:
      #print("NO SUCCESS")
      #print('Total no. of State Explored before Termination',len(state_explored))
      break
  return flag,state_explored



In [13]:
random_try=1
st=0
while random_try <= 100 and st==0:
  matrix_1d=[1,2,3,4,5,6,7,8,'B']
  random.shuffle(matrix_1d)
  x=b_to_zero(matrix_1d)
  z=np.array(x)
  start_state=z.reshape(3,3)

  target_state=[1,2,3,4,5,6,7,8,'B']
  y=b_to_zero(target_state)
  r=np.array(y)
  goal_state=r.reshape(3,3)

  is_puzzle_solvable=is_solvable(x)
  if is_puzzle_solvable=='Yes':
    print(f'\n --Random Try-{random_try}')

    print("The Initial state of puzzle is :")
    p_print(matrix_1d)

    print('Goal state of puzzle is :')
    p_print(target_state)
    print("Is this puzzle solvable : YES")
    flag,state_explored=hill_climbing(start_state,goal_state,'h1')
    if flag==1:
      print('SUCCESS')
      print('Total no. of State Explored : ',len(state_explored))
      print('Final Path is:')
      for k in state_explored:
        print(k,'\n')
      st=1
    elif flag==0:
      print("NO SUCCESS")
      print('Total no. of State Explored before Termination',len(state_explored))
    random_try +=1


 --Random Try-1
The Initial state of puzzle is :
 3 | 0 | 2
-----------
 6 | 8 | 5
-----------
 1 | 4 | 7 

Goal state of puzzle is :
 1 | 2 | 3
-----------
 4 | 5 | 6
-----------
 7 | 8 | 0 

Is this puzzle solvable : YES
NO SUCCESS
Total no. of State Explored before Termination 2

 --Random Try-2
The Initial state of puzzle is :
 1 | 4 | 7
-----------
 2 | 8 | 5
-----------
 0 | 3 | 6 

Goal state of puzzle is :
 1 | 2 | 3
-----------
 4 | 5 | 6
-----------
 7 | 8 | 0 

Is this puzzle solvable : YES
NO SUCCESS
Total no. of State Explored before Termination 1

 --Random Try-3
The Initial state of puzzle is :
 1 | 0 | 2
-----------
 8 | 5 | 7
-----------
 4 | 6 | 3 

Goal state of puzzle is :
 1 | 2 | 3
-----------
 4 | 5 | 6
-----------
 7 | 8 | 0 

Is this puzzle solvable : YES
NO SUCCESS
Total no. of State Explored before Termination 2

 --Random Try-4
The Initial state of puzzle is :
 3 | 0 | 7
-----------
 6 | 5 | 8
-----------
 4 | 2 | 1 

Goal state of puzzle is :
 1 | 2 | 3
--